# Interactive Learning Chatbot with Gradio Interface

### Import necessary libraries

In [ ]:
import json  # For handling JSON data
from typing import Dict, Any, Optional  # For type annotations
import os  # For interacting with the operating system
import gradio as gr  # Gradio library for creating the interface
from typing import List, Optional  # Additional typing utilities
from difflib import get_close_matches  # Library for string matching


<h3 id="Define a function to load a knowledge base from a JSON file">Define a function to load a knowledge base from a JSON file.</h3>
<p>
If the file doesn't exist or is empty, return an empty structure.<br>
Otherwise, load and return the contents of the JSON file as a dictionary.</p>

In [2]:
def load_knowledge_base(filename: str) -> Dict[str, Any]:
    if not os.path.exists(filename) or os.stat(filename).st_size == 0:
        return {"questions": []}
    with open(filename, 'r') as file:
        return json.load(file)

<h3 id="Define a function to save knowledge base from a JSON file">Define a function to save knowledge base from a JSON file.</h3>
<p>
Serialize the knowledge_base dictionary to JSON format with indentation for readability<br>
and write it to the specified filename..</p>

In [3]:
def save_knowledge_base(filename: str, knowledge_base: Dict[str, Any]):
    with open(filename, 'w') as file:
        json.dump(knowledge_base, file, indent=2)

<h3 id="Define a function to find the best match for a user question in a list of questions.">Define a function to find the best match for a user question in a list of questions..</h3>
<p>
Uses difflib's get_close_matches to find the closest match to user_question in questions.<br>
Returns the best match if found, otherwise returns None</p>

In [6]:
def find_best_match(user_question: str, questions: List[str]) -> Optional[str]:
    matches: List[str] = get_close_matches(user_question, questions, n=1, cutoff=0.6)
    return matches[0] if matches else None

<h3 id="Define a function to retrieve the answer for a specific question from the knowledge base.">Define a function to retrieve the answer for a specific question from the knowledge base.</h3>
<p>
Searches through the questions in knowledge_base and returns the answer corresponding to the question if found.<br>
Returns None if the question is not found in the knowledge base.</p>

In [7]:
def get_answer_for_question(question: str, knowledge_base: Dict[str, Any]) -> Optional[str]:
    for q in knowledge_base["questions"]:
        if q["question"] == question:
            return q["answer"]
    return None

<h3 id="Define a function for the chatbot that interacts based on user input and updates knowledge base.">Define a function for the chatbot that interacts based on user input and updates knowledge base.</h3>
<p>
Loads knowledge base from 'knowledge_base.json', checks if user wants to quit, finds best match<br>
for user input in existing questions, retrieves answer if found, or learns a new response if no match.<br>
Updates and saves knowledge base if new response is provided and not skipped.</p>

In [8]:
def gradio_chatbot(user_input: str, new_answer: str = "") -> str:
    knowledge_base: Dict[str, Any] = load_knowledge_base('knowledge_base.json')
    
    if user_input.lower() == 'quit':
        return 'Goodbye!'
    
    best_match = find_best_match(user_input, [q["question"] for q in knowledge_base["questions"]])
    
    if best_match:
        answer = get_answer_for_question(best_match, knowledge_base)
        return f'Bot: {answer}'
    else:
        if new_answer.lower() != 'skip' and new_answer:
            knowledge_base["questions"].append({"question": user_input, "answer": new_answer})
            save_knowledge_base('knowledge_base.json', knowledge_base)
            return 'Bot: Thank you! I learned a new response!'
        else:
            return 'Bot: I don\'t know the answer. Can you teach me?'

### Creating a Gradio interface for the chatbot using gr.Interface.

In [ ]:
iface = gr.Interface(
    fn=gradio_chatbot,  # Function to be called on user input
    inputs=["text", "text"],  # Two text input fields for user question and new answer
    outputs="text",  # Output field for displaying bot's response
    title="Chatbot with Learning Capability",  # Title displayed on the interface
    description="Enter your question in the first input. If the bot doesn't know the answer, "
                "you can teach it by providing an answer in the second input. Type 'quit' to exit."  # Description of how to use the chatbot
)

In [ ]:
iface.launch()